# Глубокое машинное обучение 

## HW1: Computer Vision

Дата *мягкого* дедлайна: `22.10.2025 23:59`

Дата **жесткого** дедлйна: `29.10.2025 23:59`

### **Правила сдачи (обязательно к прочтению)**
- После **жесткого** дедлайна работы не принимаются.
- После мягкого дедлайна каждый день снимается 10 баллов.
- Все похожие друг на друга работы оцениваются в 0.
- Если вы использовали какой-то публичный код, то это необходимо указать в комментарии и приложить ссылку на источник, иначе тоже может быть 0. :(
- За нечитаемый код, неэффективную реализацию и _некрасивые_ визуализации могут быть сняты баллы.
- Все вычисления должны быть воспроизводимые, поэтому не забывайте про `seed=42`, а также не забудьте перезапустить ноутбук и прожать все ячейки - ошибок быть не должно.
- Готовый ноутбук со всеми данными необходимо положить в архив. В архив необходимо полдожить ваш requirments.txt со всеми версиями библиотек.
- Архив называется следующим образом: `HW1_<your_latin_name>.zip`. Пример: `HW1_malkov.zip`
- Для всех трейнов важно предоставить графики лосов/эпох. Легче всего оставить ссылку на wandb, если такой возможности нет, то делайте графики сами.
- Использовать LLM можно, но допускается не более 50% сгенерированного кода в каждом задании. В случае использования LLM нужно комментарием указать модель, промт и источник инференса. Если нет комментария или количество сгенерированного кода больше 50%, то за задание ставится 0. Пример:
```python
# LLM START
"""
Model: Qwen3-30B-A3B-FP8 (указать квантизацию и кастомные параметры (top-p, top-p, temperature, ...), если есть)
Source: vLLM (здесь может быть API (openrouter.ai), может быть сайт с чатиком (chat.z.ai), может быть тула для локального инференса (lm-studio), может быть TG-бот (@jadvebot))
Prompt: Мой китайский друг, представь, что ты Ян Ле Кун, придумай за меня алгоритм детекции тачулечек
"""
print("Hello world")
# LLM END
```


### **Правила оценки работы**
Всего за работу можно получить 50 баллов. Вся домашняя работа делится на Advanced (для сильных) и Regular _(для нормальных)_, каждый из разделов дает по 25 баллов. Только выполнив Advanced задачи, можно получить **отл**. Чтобы получить **хор** (5), достаточно полностью выполнить все Regular задания.
Вот как выглядит отображение баллов в оценки:
| [От и До) | Оценка |
|-----------|--------|
| 0-5       | 1      |
| 5-10      | 2      |
| 10-15     | 3      |
| 15-20     | 4      |
| 20-26     | 5      |
| 26-30     | 6      |
| 30-35     | 7      |
| 35-40     | 8      |
| 40-45     | 9      |
| 45-50     | 10     |

### Предисловие
Вы уже узнали столько, чтобы попробовать себя в роли настоящего инженера или даже ресерчера. Сегодня мы с нуля пройдем весь путь создания детектора. Наша задача будет заключаться в том, чтобы подготовить модель для детекции медицинских масок на изображении.\

![FaceMask](https://user-images.githubusercontent.com/49322948/159162461-4552eee3-27db-49b2-ab6c-718106adc3c7.jpg)

### 0. Импорт всего, что нужно, и определение всех констант

In [1]:
import cv2
import random
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

from torchvision.models.detection import ssdlite320_mobilenet_v3_large
from torchvision.models.detection.ssdlite import SSDLiteClassificationHead, SSDLiteRegressionHead

BATCH_SIZE = 8
IMG_SIZE = 300
SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

ModuleNotFoundError: No module named 'cv2'

### (reg, 4.0) 1. Разметка

Здесь нет ссылки на датасет, потому что в реальной жизни данные вам, скорее всего, никто не даст. Вам нужно самим найти все необходимые данные и преобразовать их к нужному виду.

Готовую разметку необходимо подготовить в формате __Pascal VOC__. 

__Ваш датасет нужно запаковать в архив и отправить вместе с работой!__

In [ ]:
# https://humansintheloop.org/resources/datasets/medical-mask-dataset/

### 2. Подготовка датасета
О, ну вот и задание на код. Всё просто, нужно подготовить `dataset` и `loader`, которые будут использоваться для трейна и теста.

#### (reg, 1.0) 2.1 Определение датасета.


In [101]:
class MyVOCDataset(Dataset):
    def __init__(self, *args):
        # YOUR CODE HERE
        pass

    def __len__(self):
        # YOUR CODE HERE
        return 

    def __getitem__(self, idx):
        # YOUR CODE HERE
        return image, target

#### (reg, 1.0) 2.2 Подготовка аугументации
Аугументацию необходимо делать с помощью `albumentations`.

In [102]:
# YOUR CODE HERE
train_transforms = A.Compose([])
# Да-да, для теста аугументация не нужна, только базовые преобразования
test_transforms = A.Compose([])

#### 2.3 Cобираем всё вместе

In [103]:
# YOUR CODE HERE

# train_dataset =
# train_loader = 

# test_dataset =
# test_loader =

#### (reg, 0.5) 2.4 Визулизация датасета
Визуализируйте пару примеров, чтобы убедиться, что вы нигде не ошиблись в разметке.

In [104]:
# YOUR CODE HERE

### 3. Обучение претрейн модели
Очень вам завидую, ведь это ваш первый настоящий трейн!

В прошлом курсе вы уже обучали бустинги и, может даже, с какими-нибудь сложными `hyperopt` или `gridsearch`, но поверьте — это совсем не то. В этом задании вам пригодится абсолютно всё, что мы к этому времени прошли, + мощности colab/kaggle/... + много времени и терпения. А всё почему? А всё потому, что мы будем с вами реализовывать **SSD** с нуля без каких-либо претрейнов и без VGG в бэкбоне (это всё-таки пережиток прошлого). Ну и сравним, что у нас получится с реализацией из `torchvision`, если всё сойдется.

#### 3.0 Теоретическая вводная.

Мы уже обсуждали на лекции, что такое SSD, и если кратко, то это One-Stage детектор, аналогичный YOLO, в котором есть несколько трюков:
1. Берут готовый бэкбон (VGG16), а не обучают его с нуля на ImageNet.
2. Вместо FC слоев используют только свертки. (прям как в RPN Faster-RCNN)
3. Из оригинальной VGG убирают все полносвязные слои и добавляют дополнительно 5 слоев сверток.
4. Вместо одного последнего слоя/активации (как в YOLO) берут 6 слоев из разных мест сети и «делают на них предсказания». Нужно это для того, чтобы лучше работать с объектами маленьких размеров, с которыми YOLO работает плохо.

Если более простыми словами объяснять проблематику из пункта 4, то в случае YOLO мы после всех преобразований оригинального изображения 448х448(x3) мы получаем карту активации 7x7(x1024). Таким образом, для каждого 64x64 квадрата оригинального изображения мы имеем всего 1x1x1024 фичи, характеризующий каждый из квадратов 64x64, при этом пространственной информации о содержании этих квадратов там почти нет. Зато пространственной информации будет больше на более ранних картах активации, например, размера 32х32, где каждый «элемент сетки» уже будет отвечать за квадрат 14x14 оригинального изображения. При этом, очевидно, «фичи» из карты активации 32х32 будут «слабее» фичей из 7х7, просто потому что слой 7х7 глубже. 

![SSD](https://www.acetrace.app/images/illustrations/ssd.png)

Если всё ещё не очень понятно, что в этом вашем SSD происходит, то обязательно посмотрите вот на этот реп: [клик](https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Object-Detection).

#### 3.1 Практическая вводная
Первое, что вам нужно сделать, — это прочитать [статью](https://arxiv.org/pdf/1512.02325) с вниманием ко всем деталям. Оттуда, например, нужно заимствовать формулки для расчета скейла: $s_k = s_{min} + \frac{s_{max}-s_{min}}{m-1}(k-1)$ и узнать про _hard negative mining_ (все нюансы из статьи правда важны, её же не глупые люди писали!).

Какие-то детали с точки зрения имплементации можно подчерпнуть из:
- [официальной имплементации](https://github.com/weiliu89/caffe/blob/2c4e4c2899ad7c3a997afef2c1fbac76adca1bad/examples/ssd/ssd_coco.py).
- [популярной учебной имплементации](https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Object-Detection/blob/master/model.py).
- [имплементации команды torch](https://github.com/pytorch/vision/blob/main/torchvision/models/detection/ssd.py).
- [просто чья-то имплементация](https://github.com/amdegroot/ssd.pytorch/tree/master).

Также очень рекомендую прочитать блог команды torch относительно имплементации SSD и SSD-Lite (это SSD с MobileNet в бэкбоне):
- [SSD](https://pytorch.org/blog/torchvision-ssd-implementation/).
- [SSD-Lite](https://pytorch-hub-preview.netlify.app/blog/torchvision-ssdlite-implementation/).

#### **3.2 План действий (обязательно к прочтению)**
Теперь по поводу плана. Если вы не GPU-rich и не имеете доступа к серверу хотя бы с 4 GPU, то у вас вряд ли получится сделать претрейн SSD с нуля, чтобы она прям была пригодна для использования, поэтому мы с вами сфокусируемся на самой архитектуре и на непосредственно процессе обучения. Пункт с обучением своей модели будет выполнен, если:
1. Всё в вашей имплементации будет по делу.
2. Ваша имплементация будет спокойно **переобучаться** на одном батче за 1000 итераций. Это значит, что вы все правильно имплементировали.
3. У вас будет хотя бы **10-20 эпох**, где видно, что лосс классификации и лосс регрессии уверенно сходятся. При этом вам нужно показать минимум **пять** конфигураций обучения (с разными гиперпараметрами/лосами/оптимизациями/архитектурными триками). 

Если вы все сделали всё правильно, то на ваших графиках loss-а вы увидите что-то такое:
<div>
<img src="https://d3e0luujhwn38u.cloudfront.net/resized/kz9vKVmfahkZKS5UC_tJ8SJVHjeHIGDO8X-wiz8PzU8/s:3200/plain/s3://typefully-user-uploads/img/original/36777/1e59208e-44c4-4ed1-9c13-ee0010afc4cb.png" width="500"/>
</div>

Теперь пару советов с точки зрения самой реализации:
1. Бэкбон предлагаю брать максимально компактный типа MobileNetV3.
2. Предлагаю не брать какой-то большой датасет типа [COCO](https://public.roboflow.com/object-detection/microsoft-coco-subset/2), а ограничится [PASCAL VOC](https://public.roboflow.com/object-detection/pascal-voc-2012/1).
3. Здорово, если вы в какой-то имплементации увидите, что авторы используют SGD, а в другой использует какие трики с масштабированием дисперсии, но будьте уверены, в вашем случае это может не работать или работать не очень хорошо. Нужно все проверять самостоятельно.  

Куда обязательно нужно посмотреть: 
1. Уменьшения изображения — это ок для нашего домашнего претрейна.
2. Инициализация весов — это очень важно.
3. Сильный дисбаланс классов надо в любом случае компенсировать.

И несколько практических советов:
1. Обязательно делайте чекпоинты во время трейна.
2. Старайтесь сделать код максимально гибким и удобным для ваших экспериментов, а не копипастить одну и ту же функцию трейна с разными оптимизаторами.
3. Внимательно смотрите не только на значения лосов по эпохам, но по всем итерациям/степам — это сэкономит вам время.
4. Обязательно логируйте все эксперименты в тексте/тензорборде/wandb и смотрите графики. Не забывайте делать оценку на val-выборке, особенно если используете не очень большой датасет типа PASCAL.

#### (adv, 5.0) 3.3 Определение модели
VGG, скорее всего, очень сильно отличается по архитектуре от выбранного вами бэкбона, поэтому вам самостоятельно нужно подобрать слои, из которых брать фичи, а также определить количество анкеров и базовые соотношения сторон боксов.

Для того чтобы долго не высчитывать, какой размер сверток вам нужен на каждом этапе, лучше просто посмотреть, какие шейпы получаются на каждом слое. Вот пример кода:
```python
backbone = torchvision.models.get_model("mobilenet_v3_large", weights="DEFAULT").features
x = torch.randn(1, 3, IMAGE_SIZE, IMAGE_SIZE)
for i, layer in enumerate(backbone):
    x = layer(x)
    print('back', i, x.shape)

extra_layers = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(960, 256, 1, bias=False), nn.BatchNorm2d(256), nn.ReLU6(inplace=True),
                SeparableConv2d(256, 512, kernel_size=3, stride=2, padding=1)
            ),
            nn.Sequential(
                nn.Conv2d(512, 128, 1, bias=False), nn.BatchNorm2d(128), nn.ReLU6(inplace=True),
                SeparableConv2d(128, 256, kernel_size=3, stride=2, padding=1)
            ),
        ])

for i, layer in enumerate(extra_layers):
    x = layer(x)
    print('ext', i, x.shape)
```

In [105]:
class MySSD(nn.Module):
    def __init__(self, *args):
        super().__init__()
        # YOUR CODE HERE 

    def _generate_default_boxes(self, *args, s_min=0.1, s_max=0.9):
        """
        Функция генерации дефолтных боксов, как из статьи
        """
        # YOUR CODE HERE
        return

    def forward(self, x):
        # YOUR CODE HERE
        return

#### (adv, 2.0) 3.4 Определение лоса
В оригинальной статье лосс состоит из двух компонет: лосс регрессии (L1_smooth) и лосс классификации (кросс-энтропия) с трюком Hard negative mining.

In [106]:
class MySSDLoss(nn.Module):
    def __init__(self, *args):
        super().__init__()
        # YOUR CODE HERE
        
    def forward(self, *args):
        # YOUR CODE HERE
        return conf_loss + self.alpha * loc_loss, conf_loss, loc_loss

#### (adv, 1.0) 3.5 Аугументация для претрейна

In [107]:
# YOUR CODE HERE
pre_train_transforms = A.Compose([])
pre_val_transforms = A.Compose([])

#### (adv, 1.0) 3.6 Подготовка датасета и лоудера для претрейна
> Нужно обязательно переисользовать `MyVOCDataset`.

In [108]:
# YOUR CODE HERE

# pre_train_dataset =
# pre_train_loader = 

# pre_val_dataset =
# pre_val_loader =

# pre_test_dataset =
# pre_test_loader =

#### (reg, 2.0) 3.7 Визулизация датасета
Визуализируйте пару примеров, чтобы убедиться, что вы нигде не ошиблись в разметке.

In [109]:
# YOUR CODE HERE

#### (adv, 1.0) 3.8 Подготовка train-loop для переобученя на одном батче
> Не забывайте про перевод модели в режим трнйна (`model.train()`) или в режим инференса (`model.eval()`), а также помните про контекстный менджер: (`torch.no_grad()`).

In [110]:
def run_single_batch_overfit(*args, iterations=1500):
    # YOUR CODE HERE
    pass

#### (adv, 2.0) 3.9 Оверфит на одном батче

In [111]:
# YOUR CODE HERE

#### (adv, 1.0) 3.10 Визуализация экспериментов

In [112]:
# YOUR CODE HERE OR LINK TO WANDB OR ANY VISUALIZAION

#### (adv, 1.0) 3.11 Подготовка train-loop для обучения претрейна


In [113]:
def run_training_stage(*args,):
    # YOUR CODE HERE
    pass

#### (adv, 8.0) 3.12 Обучение претрейна
> Не нужно ждать, что все сойдется, достаточно просто показать 5 разных конфигураций обучения с 20-30 эпохами, где лосс _уверенно_ падает.

In [114]:
# YOUR CODE HERE

#### (adv, 1.0) 3.13 Визуализация экспериментов

In [115]:
# YOUR CODE HERE OR LINK TO WANDB OR ANY VISUALIZAION

Если вы дошли до этого пункта, то вы просто герой! („• ֊ •„)੭

По секрету: если на собеседовании вы скажете, что вы _Senior AI Developer_, то никто даже не поймет подвоха.

### 4. Обучение основной модели на базе готового претрейна
В этом блоке мы зафайнтюним предобученную SSD-Lite на нашей собственной разметке.

#### (reg, 1.0) 4.1 Импортируйте веса модели
Вот сразу ссылка на сурсы, если что: [клик](https://github.com/pytorch/vision/blob/main/torchvision/models/detection/ssdlite.py). Для вас задача ещё сложнее: дефолтно в [SSD](https://github.com/pytorch/vision/blob/7a13ad0f89167089616b51f4fd07f978cf1f17e4/torchvision/models/detection/ssd.py#L453) и [SSD-Lite](https://github.com/pytorch/vision/blob/7a13ad0f89167089616b51f4fd07f978cf1f17e4/torchvision/models/detection/ssdlite.py#L310) включен NMS, а нам надо его выключить, потому что мы сделаем его сами. Напомнию, что NMS на этапе обучения __[не нужен](https://github.com/pytorch/vision/blob/7a13ad0f89167089616b51f4fd07f978cf1f17e4/torchvision/models/detection/ssd.py#L404)__.

In [129]:
model = ssdlite320_mobilenet_v3_large(weights="DEFAULT")

# YOUR CODE HERE

# model.to(DEVICE)

#### (reg, 4.0) 4.1 Подготовьте train-loop

In [117]:
def train_ssd(*args,):
    # YOUR CODE HERE
    pass

#### (reg, 3.0) 4.2 Обучите модель на нашем датасете

In [118]:
# YOUR CODE HERE

#### (reg, 2.0) 4.3 Визуализация эксперимента

In [119]:
# YOUR CODE HERE OR LINK TO WANDB OR ANY VISUALIZAION

#### (reg, 2.0) 4.4 Реализация NMS
Мы уже обсуждали алгоритм NMS на лекции, но если забылось, то вот отличная [статья](https://deepmachinelearning.ru/docs/Neural-networks/Object-detection/Non-maximum-supression).

In [120]:
def nms(boxes, scores_max, iou_threshold):
    # YOUR CODE HERE
    pass

#### (reg, 2.0) 4.5 Реализация функции декодирования предсказаний
Нужно реализовать функцию для переовода offset + default_box/anchors, которые выдает модель, в xyxy bbox.

In [121]:
def decode_boxes(box_preds, default_boxes):
    # YOUR CODE HERE
    pass

#### (reg, 0.5) 4.5 Визализация работы модели на val датасете
Выведит пару примеров детекции масок

In [122]:
# YOUR CODE HERE

#### (reg, 2.0) 4.6 Подсчет IoU
Для того чтобы оценить качество работы самой детекции чаще всего измеряют IoU. Пропустите вашу модель чере NMS и посчитайте IoU отонсительно Ground Truth. Вычисление IoU нужно реализовать самостоятелньо!

In [123]:
def calculate_iou(boxs1, boxs2):
    # YOUR CODE HERE
    pass

# YOUR CODE HERE

#### (reg, 2.0) 4.7 Подсчет mAP
Про `Mean Average Precision` вы точно слышали из прошлого курса. Считается немного модифицированный `PR AUC` по всем наблюдениям одного класса (у нас он только один, фон не считается) и получаем `AP`, а дальше усредняем `AP` по всем классам и получаем `mAP`. [Тут](https://learnopencv.com/mean-average-precision-map-object-detection-model-evaluation-metric/) пример на пальцах. Расчет mAP тоже нужно реализовать самостоятельно.

In [124]:
def map(*args, iou_threshold=0.5, score_threshold=0.01):
    # YOUR CODE HERE
    pass

# YOUR CODE HERE

### 5. Обучение модели на базе собственного претрейна
**Все пункты дальше необязательные! И их можно делать, только если у вас хоть чуть-чуть сошелся лосс и модель классифицирует объекты лучше рандома!**

#### 5.1 Дообучение вашего претрейна 

In [125]:
# YOUR CODE HERE

#### 5.2 Визуализация эксперимента

In [126]:
# YOUR CODE HERE

#### 5.3 Подсчет IoU

In [127]:
# YOUR CODE HERE

#### 5.3 Подсчет mAP

In [128]:
# YOUR CODE HERE